<a href="https://colab.research.google.com/github/akshithamaddi/Akshitha_INFO5731_Spring2020/blob/master/Copy_of_Copy_of_Copy_of_Copy_of_Copy_of_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import glob
import pandas as pd
import os
import csv
# Select all the files in your directory
directory = r'/content/drive/My Drive/sigir_data'
files = glob.glob(directory + "/*.csv")
df1 = pd.concat([pd.read_csv(fp,skipinitialspace=True,encoding = "ISO-8859-1").assign(New=os.path.basename(fp)) for fp in files])
df1['article_no'] = df1['article_id'].str.replace('https://doi.org/','')
df1.to_csv('output.csv')

In [0]:
data = pd.read_csv("output.csv") 
df1 = data[pd.notnull(data['article_id'])]
df1.to_csv('articleoutput.csv')
#reading the csvfile
data = pd.read_csv("articleoutput.csv") 
header = ["article_no"]
df1.to_csv('articleno.csv', columns = header)

In [0]:
import csv
import itertools
contents = []
with open('articleno.csv','r') as csvf:
    urls = csv.reader(csvf)
    next(urls, None)
    contents = [row[1] for row in itertools.islice(urls,768)] 
string = 'https://api.semanticscholar.org/v1/paper/'
my_new_list = [string+x for x in contents]
with open("Article_URLs.csv","w") as f:
    writer = csv.writer(f,delimiter="\n")
    writer.writerow(['ArticleURL'])
    writer.writerow(my_new_list) 

In [0]:
import urllib.request
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import time
countUp = 0
headers = {}
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17'
for j in my_new_list:
 time.sleep(2)
 try:
     req = urllib.request.Request(j,headers=headers)
     response = urllib.request.urlopen(req)
     res=response.read().decode('utf8')
     time.sleep(0.1)
     print("Here")
     with open ('/content/jsonfiles/Article' + str(countUp) + '.json', 'w') as g:
          g.write(res)
     countUp += 1
 except HTTPError as e:
    print('The server couldn\'t fulfill the request.')
    print('Error code: ', e.code)
    time.sleep(30) 
 except URLError as e:
    print('We failed to reach a server.')
    print('Reason: ', e.reason)
    time.sleep(30)    

In [0]:
from google.colab import files
#For the folder you have to zip it first and can only download later on
!zip -r jsonfiles.zip /content/jsonfiles
#Download files
files.download('jsonfiles.zip')

In [0]:
!pip install textblob
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import word_tokenize, pos_tag, pos_tag_sents
from textblob import Word
from nltk.stem import PorterStemmer

nltk.download('stopwords')
st=PorterStemmer()
stop=stopwords.words('english')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
#table 1
import json
import pandas
from pandas import DataFrame
Venuelist = []
Titlelist = []
Yearlist = []
Abstractlist = []
Doilist = []
FieldsOfStudylist = []
Topiclist = []
urllist = []
influentialCitationCountlist = []
articleidlist = []
for i in range(968):
  f = open('/content/jsonfiles/Article' + str(i) + '.json')
  data = json.load(f)
  string = 'https://doi.org/'
  articleidlist = [string+x for x in Doilist]
  venue = data['venue']
  Venuelist.append(venue)
  title = data['title']
  Titlelist.append(title)
  year = data['year']
  Yearlist.append(year)
  abstract = data['abstract']
  Abstractlist.append(abstract)
  doi = data['doi']
  Doilist.append(doi)
  fieldsOfStudy = data['fieldsOfStudy']
  FieldsOfStudylist.append(fieldsOfStudy)
  influentialCitationCount = data['influentialCitationCount']
  influentialCitationCountlist.append(influentialCitationCount)
  for value in data['topics']:
    topic = value['topic']
    Topiclist.append(topic)
  url = data['url']
  urllist.append(url)   
f.close()
#print(Topiclist)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
import pandas as pd
import glob
import pandas as pd
import os
import string

# Select all the files in your directory
fields = ["article_id","pdf_link","total_citation","total_download","pdf_link","article_no"]
dfarticle = pd.read_csv("articleoutput.csv", usecols=fields)
doilist = dfarticle['article_no'].values.tolist()
articleidlist = dfarticle['article_id'].values.tolist()
totalcitationlist = dfarticle['total_citation'].values.tolist()
totaldownloadlist = dfarticle['total_download'].values.tolist()
pdflinklist = dfarticle['pdf_link'].values.tolist()
a = {'Article_id' : articleidlist ,'Venue' : Venuelist ,'Title' : Titlelist ,'Year' :Yearlist, 'Abstract': Abstractlist,
     'Doi' : doilist,'fieldsOfStudy' : FieldsOfStudylist,'Downloadurl' : pdflinklist,
     'TotalCitationCount' : totalcitationlist, 'TotalDownloadCount' : totaldownloadlist, 'Influentialcitationcount' : influentialCitationCountlist}
df = pd.DataFrame.from_dict(a, orient='index')
#df['fieldsOfStudy'].str.strip('[]').astype(int)
df1 = df.transpose()
#df1['fieldsOfStudy'] = df1['fieldsOfStudy'].str.strip('[]')
df1["Abstract"]=df1["Abstract"].str.lower() 
df1['Abstract'] = df1['Abstract'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))
df1["Abstract"]=df1['Abstract'].str.replace('[{}]'.format(string.punctuation), '')
df1['fieldsOfStudy'] = df1['fieldsOfStudy'].apply(str)
df1['fieldsOfStudy'] = df1['fieldsOfStudy'].str.replace('[{}]'.format(string.punctuation), '')
df1['fieldsOfStudy']
df1.to_csv('/content/jsonfiles/table1.csv', columns = a)
print(df1)

                                Article_id  ... Influentialcitationcount
0    https://doi.org/10.1145/511285.511286  ...                        0
1    https://doi.org/10.1145/511285.511288  ...                        0
2    https://doi.org/10.1145/511285.511289  ...                        0
3    https://doi.org/10.1145/511285.511290  ...                        0
4    https://doi.org/10.1145/511285.511292  ...                        0
..                                     ...  ...                      ...
970  https://doi.org/10.1145/345508.345671  ...                     None
971  https://doi.org/10.1145/345508.345672  ...                     None
972  https://doi.org/10.1145/345508.345673  ...                     None
973  https://doi.org/10.1145/345508.345674  ...                     None
974  https://doi.org/10.1145/345508.345675  ...                     None

[975 rows x 11 columns]


In [0]:
#table 2
import json
import pandas
from pandas import DataFrame
authorIdlist = []
authorname = []
authorurl = []
doilist = []
#authorrank = []
#authorinstitution = []
#authorPublication = []
for i in range(968):
  f = open('/content/jsonfiles/Article' + str(i) + '.json')
  data = json.load(f)
  string = 'https://doi.org/'
  articleidlist = [string+x for x in Doilist]
  doi = data['doi']
  Doilist.append(doi)
  for value in data['authors']:
    authorId = value['authorId']
    authorIdlist.append(authorId) 
  for value in data['authors']:
    name = value['name']
    authorname.append(name) 
  for value in data['authors']:
    url = value['url']
    authorurl.append(url)
f.close()

In [0]:
import pandas as pd
import glob
import pandas as pd
import os
# Select all the files in your directory
a = {'authorId' : authorIdlist ,'name' : authorname ,'url' : authorurl }
df = pd.DataFrame.from_dict(a, orient='index')
df1 = df.transpose()
df1.to_csv('/content/jsonfiles/table2.csv', columns = a)

In [0]:
#table 3
import json
import pandas
from pandas import DataFrame
citingpaperId = []
citingdoi = []
citingtitle = []
citingvenue = []
citingyear = []
citingintent = []
citinginfluential = []
for i in range(968):
  f = open('/content/jsonfiles/Article' + str(i) + '.json')
  data = json.load(f)
  for value in data['citations']:
    paperId = value['paperId']
    citingpaperId.append(paperId)
    #print(paperId)
    doi = value['doi']
    citingdoi.append(doi)
    #print(doi)
    title = value['title']
    citingtitle.append(title)
    #print(title)
    venue = value['venue']
    citingvenue.append(venue)
    #print(venue)
    year = value['year']
    citingyear.append(year)
    #print(year)
    intent = value['intent']
    citingintent.append(intent)
    #print(intent)
    isInfluential = value['isInfluential']
    citinginfluential.append(isInfluential) 
    #print(isInfluential)
a = {'paperId' : citingpaperId ,'doi' : citingdoi ,'title' : citingtitle ,
     'venue' :citingvenue, 'year': citingyear,
     'intent' : citingintent,'isInfluential' : citinginfluential}
df = pd.DataFrame.from_dict(a, orient='index')
df1 = df.transpose()
df1['intent'] = df1['intent'].apply(lambda x : ' '.join(map(str, x)))
df1.to_csv('/content/jsonfiles/table3.csv', columns = a)
print(df1)

                                        paperId  ... isInfluential
0      8113edc6c95508bc7f9ae624903f8f4c98d3395b  ...         False
1      819c14aeb7fbd7819baa58f038142db1f0cfc351  ...         False
2      28dd43a10b78cc6297081fb7b9be86e114894391  ...         False
3      24f73ab2828f9aecbc079a9d5f5028c392eaa5b0  ...         False
4      393a9de965783f2c0f42b82db47321df10402472  ...         False
...                                         ...  ...           ...
65632  e85206cb8c17461f8f823972a44a33b2eaed1ddb  ...         False
65633  4665b979d409e9878404ead82e87d787cbca270f  ...         False
65634  c022253b7988d517a53f48edf5867b3b5557271d  ...         False
65635  d4ce5e9394ecd2cb3e64bb952885a236e99f1fc5  ...         False
65636  04ab5ed3635a75bfd6b0c95598dbf38a1d4f63e8  ...         False

[65637 rows x 7 columns]


In [0]:
#table 4
import re
referencepaperidlist = []
Referencepaperdoilist = []
Referencepapertitlelist = []
Referencepapervenuelist = []
Referencepaperyearlist = []
Isinfluentiallist = [] 
Intentlist = []
for i in range(968):
  f = open('/content/jsonfiles/Article' + str(i) + '.json')
  data = json.load(f)
  for value in data['references']:
    referencepaperid = value['paperId']
    referencepaperidlist.append(referencepaperid)
    referencepaperdoi = value['doi']
    Referencepaperdoilist.append(referencepaperdoi)
    Referencepapertitle = value['title']
    Referencepapertitlelist.append(Referencepapertitle)
    Referencepapervenue = value['venue']
    Referencepapervenuelist.append(Referencepapervenue)
    Referencepaperyear = value['year']
    Referencepaperyearlist.append(Referencepaperyear)
    Intent = value['intent']
    Intentlist.append(Intent)
    Isinfluential = value['isInfluential']
    Isinfluentiallist.append(Isinfluential)
a = {'Reference_paper_id' : referencepaperidlist ,'Reference _paper_doi' : Referencepaperdoilist ,'Reference _paper_title' : Referencepapertitlelist ,
     'Reference _paper_venue' :Referencepapervenuelist, 'Reference _paper_year': Referencepaperyearlist,
     'Intent' : Intentlist,'Is_influential' : Isinfluentiallist}
df = pd.DataFrame.from_dict(a, orient='index')
df1 = df.transpose()
df1['Intent'] = df1['Intent'].apply(lambda x : ' '.join(map(str, x)))
df1.to_csv('/content/jsonfiles/table4.csv', columns = a)
print(df1)


                            Reference_paper_id  ... Is_influential
0     cb93b7894a19780851dcaf0559a8c4f9ac1a7694  ...          False
1     6f7949a0301963ca49d88c5c96b375ded713a31b  ...          False
2     dccea747182423d212741a6eecabf4d2024efb52  ...          False
3     a76e8341a099a3ab447ee16aafcd24213cb0dd18  ...          False
4     848dc213c8e0d1d33dd57e91b713e1dce1a6ef1e  ...          False
...                                        ...  ...            ...
8515  8b414a17fe8142a9ff5f5af6ba63564652e7534f  ...          False
8516  9f4e1ce21458d3cd8b74c0a19dea77db3bc0575f  ...          False
8517  53d61dad3b913c7c8ac602ed82fb5445ca506648  ...          False
8518  e5305866d701a2c102c5f81fbbf48bf6ac29f252  ...          False
8519  b3294141d08b8d57e1e2d87cee1d43f6e4ef12ac  ...          False

[8520 rows x 7 columns]
